<a href="https://colab.research.google.com/github/rajatDpatil/EchoTruth---Fake-News-Detection-System/blob/main/EchoTruth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas scikit-learn Gradio joblib

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re
import joblib
import string


In [3]:
fake=pd.read_csv('/content/Fake.csv')
true=pd.read_csv('/content/True.csv')

In [4]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [6]:
fake['class']=0
true['class']=1

In [7]:
data=pd.concat([fake,true], axis=0)

In [8]:
data.sample(10)

,title,text,subject,date,class
15673,Fight for grassroots democracy in Cambodia sid...,"BATTAMBANG, Cambodia (Reuters) - Sin Rozeth s ...",worldnews,"November 5, 2017",1
13393,SHOCKER! WHY BERNIE SUPPORTERS At DNC Overwhel...,It s only logical that the people who supporte...,politics,"Jul 29, 2016",0
21372,U.N. calls for pause in air strikes to spare c...,GENEVA (Reuters) - The United Nations called o...,worldnews,"August 24, 2017",1
9684,SHERIFF CLARKE CALLS OUT NFL For Latest Move T...,Sheriff Clarke has it right! The NFL wasn t li...,politics,"Oct 14, 2017",0
1521,Deputy Attorney General PISSED; Trump Ordered...,Just when you think you re starting to figure ...,News,"May 11, 2017",0
6504,Ryan says expects legislation to replace Obama...,WASHINGTON (Reuters) - U.S. House of Represent...,politicsNews,"January 5, 2017",1
3430,Putin: Russia struck no secret agreements with...,"ST PETERSBURG, Russia (Reuters) - President Vl...",politicsNews,"June 2, 2017",1
9135,AFTER A SURPRISE DETOUR…PRESIDENT TRUMP Arrive...,PRESIDENT TRUMP MADE A SURPRISE DETOUR ON HIS ...,politics,"Dec 22, 2017",0
5275,"In sweeping move, Trump puts regulation monito...",WASHINGTON (Reuters) - President Donald Trump ...,politicsNews,"February 24, 2017",1
18734,UK's May urges EU to respond in kind to new to...,"TAPA, Estonia (Reuters) - British Prime Minist...",worldnews,"September 29, 2017",1


In [9]:
data=data.drop(["title","subject","date"],axis=1)

In [10]:
data.reset_index(inplace=True)

In [11]:
data.drop(['index'],axis=1,inplace=True)

In [12]:
data.sample(5)

,text,class
7941,"On February 20, Justin Linden went out to cauc...",0
42403,PARIS (Reuters) - France said on Wednesday it ...,1
36527,MELBOURNE (Reuters) - China accused Australia ...,1
3761,If Donald Trump picked retired General David P...,0
28302,WASHINGTON (Reuters) - Some Republican lawmake...,1


In [13]:
def clean_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove special characters and brackets (using raw strings)
    text = re.sub(r'\[.*?\]', '', text)

    # Remove backslashes
    text = re.sub(r'\\', '', text)

    # Remove URLs (improved pattern)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # Remove punctuation using string.punctuation
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)

    # Remove newlines and extra whitespaces
    text = re.sub(r'\n', '', text)

    # Remove words containing digits
    text = re.sub(r'\w*\d\w*', '', text)

    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text



In [14]:
data['text'] = data['text'].apply(clean_text)

In [15]:
x= data['text']
y= data['class']

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.25, random_state=42)

In [16]:
vectorizer = TfidfVectorizer()
xv_train = vectorizer.fit_transform(xtrain)
xv_test = vectorizer.transform(xtest)

In [17]:
lr= LogisticRegression()
lr.fit(xv_train, ytrain)

LogisticRegression()

In [18]:
predictions = lr.predict(xv_test)
lr.score(xv_test, ytest)

0.985924276169265

In [19]:
print(classification_report(ytest, predictions))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5895
           1       0.98      0.99      0.99      5330

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [20]:
joblib.dump(vectorizer,"vectorizer.jb")
joblib.dump(lr,"lr_model.jb")

['lr_model.jb']